In [1]:
import sys
sys.path.append('/media/gpt4-pdf-chatbot-langchain/dev-langchain-ChatGLM/')
from langchain.llms.base import LLM
import torch
import transformers 
import models.shared as shared 
from abc import ABC

from langchain.llms.base import LLM
import random
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList
from typing import Optional, List, Dict, Any
from models.loader import LoaderCheckPoint 
from models.base import (BaseAnswer,
                         AnswerResult)



INFO  2023-06-09 20:52:01,296-1d: 
loading model config
llm device: cuda
embedding device: cuda
dir: /media/gpt4-pdf-chatbot-langchain/dev-langchain-ChatGLM
flagging username: 35d96e513c5347dbb0c1d7c2fb21cbd4



In [2]:
import asyncio
from argparse import Namespace
from models.loader.args import parser
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
 
args = parser.parse_args(args=['--model', 'vicuna-13b-hf', '--no-remote-model', '--load-in-8bit'])

args_dict = vars(args)

shared.loaderCheckPoint = LoaderCheckPoint(args_dict)
torch.cuda.empty_cache()
llm=shared.loaderLLM() 


Loading vicuna-13b-hf...


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
/media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /media/gpt4-pdf-chatbot-langchain/pyenv-langchain did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /opt/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /media/gpt4-pdf-chatbot-langchain/pyenv-langchain/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded the model in 11.22 seconds.


In [3]:
from jsonformer import Jsonformer
json_schema = {
    "type": "object",
    "properties": {
        "action": {"type": "string"},
        "action_input": {"type": "string"}
    }
}

prompt = """ Respond to the human as helpfully and accurately as possible. You have access to the following tools:

State of Dialogue History System: 露ᥫᩣ, 张毛峰的对话 - 当需要查找露ᥫᩣ, 张毛峰之间的聊天内容时，这里面的回答是很有用的。输入应该是一个完整的问题。
Summary: useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tool_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.

Question: 我想查看关于露露的摘要信息
"""
jsonformer = Jsonformer(shared.loaderCheckPoint.model, shared.loaderCheckPoint.tokenizer, json_schema, prompt)
generated_data = jsonformer()

print(generated_data)

{'action': 'summary', 'action_input': '露ᥫᩣ,'}


In [4]:
from langchain.tools import StructuredTool

def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b

tool = StructuredTool.from_function(multiplier)

In [5]:
from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType


In [6]:


# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type. 
agent_executor = initialize_agent([tool], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [19]:
agent_executor.run("What is 1 times 14")



> Entering new AgentExecutor chain...
__call:System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

multiplier: multiplier(a: float, b: float) -> float - Multiply the provided floats., args: {{'a': {{'title': 'A', 'type': 'number'}}, 'b': {{'title': 'B', 'type': 'number'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or multiplier

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}
```

Begin! Reminder to ALWAYS respond with a valid json blob

''